# Generate Clean Text

In [ ]:
%%writefile cleantext.py
import json
with open('/users/grad/sajadi/backup/wikipedia/20160305/texts/enwiki-20160305-annonated.json') as infile , open('/users/grad/sajadi/backup/wikipedia/20160305/texts/enwiki-20160305-cleantext.json', 'w') as outfile:
    for line in infile.readlines(): 
        doc = json.loads(line)
        outfile.write(doc['title'].encode('utf-8')+'\n')
        outfile.write(doc['text'].encode('utf-8')+'\n\n')
        #out.write(line)
print "done"    

# Generate Clean Text with surface replaced with ID

In [8]:
%%writefile replace_surface.py
import re
import sys
import urllib
import sys
from HTMLParser import HTMLParser
import time

sys.path.insert(0,'..')
from memapi import memwiki as wiki

fileinput = sys.stdin
skip_line = 1; #LINE 0 is the title

def title2id(title):
    if not title:
        return "NA1"
    wid = wiki.title2id(title)
    if wid is None:
        title=title[0].upper()+title[1:]    
        wid = wiki.title2id(title)
    if wid is None:
        return "NA2"
    return str(wid)
    

def url2id(antext, url):
    hp = HTMLParser()
    
    url=url.encode('utf-8')
    url =  urllib.unquote(url)
    url = url.decode('utf-8')

    url=hp.unescape(url)
    url=hp.unescape(url)
    url=url.replace(u"\xA0"," ")
    x = url.find("#")
    if x!=-1:
        url=url[:x]
    return "id_"+title2id(url)
    
    
def replacelinks(text):
    
    annotations = []
    deltaStringLength = 0
    hrefreg=r'<a href="([^"]+)">([^>]+)</a>'
    
    text = re.sub(hrefreg, lambda m:url2id(m.group(2), m.group(1)), text)  
    return text


def process():
    line_no = -1    
    hp = HTMLParser()
    rstart=r'<doc id="(.*)" url="(.*)" title="(.*)">'
    rend=r'</doc>'
    for line in fileinput.readlines():
        line = line.decode('utf-8').strip()
        if not line:
            continue
            
        ms = re.match(rstart, line)
        if ms is not None:
            wid=ms.group(1)
            wtitle=hp.unescape(ms.group(3)).replace(u"\xA0"," ")
            line_no=0
            #print 'id_'+title2id(wtitle)
            continue
        if line_no == skip_line:
            print ""
            line_no +=1
            continue

        if re.match(rend,line):
            line_no=-1
            print "\n"
            continue    

        text = replacelinks(line).encode('utf-8')
        print text
        line_no += 1
        continue
    
if __name__ == "__main__": 
    #startTime = time.time()
    wiki.load_tables()
    #print 'wiki loaded to memory'    
    #print time.time()-startTime
    #sys.stdout.flush()
    process()

Overwriting replace_surface.py


# Usage 
time cat ~/backup/wikipedia/20160305/texts/xmltexts/wiki_* | python replace_surface.py > ~/backup/wikipedia/20160305/texts/enwiki-20160305-replace_surface.json

# Training

In [2]:
%%writefile train4.py
import sys
import logging
from gensim import *
import json
import os

in_file = "/users/grad/sajadi/backup/wikipedia/20160305/edited/enwiki-20160305-pagelinks.dumped.shuf.ssv"
#in_file = "/users/grad/sajadi/backup/wikipedia/20160305/texts/enwiki-20160305-cleantext.txt"
#in_file = "/users/grad/sajadi/backup/wikipedia/20160305/texts/enwiki-20160305-replace_surface.skip.1.txt"
#in_file = "/users/grad/sajadi/backup/wikipedia/20160305/texts/enwiki-20160305-replace_surface.txt"
output_root = "/users/grad/sajadi/backup/wikipedia/20160305/embed/"

params={"in_file_path": in_file,
         "phrasing":0,
         "sg":1, "size":500, "window":2, "min_count":5, "workers":20, "negative":0, "iter":15,
         }
baseinname=os.path.splitext(os.path.basename (params['in_file_path']))[0]
params['params_desc'] = 'word2vec.%s.%s.%s.%s.%s.%s.%s.%s.%s' % (baseinname, params["sg"], params['phrasing'],params["size"], params["window"],
                        params["min_count"], params["workers"], params["negative"],params["iter"] )
params["out_dir"] = os.path.join(output_root, params['params_desc'])
params["out_file"] = os.path.join(params["out_dir"], params['params_desc'])
if not os.path.exists(params["out_dir"]):
        os.makedirs(params["out_dir"])
with open(os.path.join(params["out_dir"], 'README.md'), 'w') as f:
    f.write(json.dumps(params))
####################################

logging.basicConfig(filename='logging.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

print "loading"   
sys.stdout.flush()
sentences = models.word2vec.LineSentence(params['in_file_path'])
if params['phrasing']==1:
    bigram_transformer = models.phrases.Phraser(models.Phrases(sentences))
    sentence_stream = bigram_transformer[sentences]
else:
    sentence_stream = sentences
print "training"    
sys.stdout.flush()
model = models.Word2Vec(sentence_stream, sg=params["sg"], size=params["size"], window=params["window"],
                        min_count=params["min_count"], workers=params["workers"], 
                        negative=params["negative"], iter=params["iter"])
model.save(params["out_file"])
print "done"    
sys.stdout.flush()


Writing train4.py
